# EazyML Counterfactual

### Install eazyml-counterfactual library

In [ ]:
!pip install --upgrade eazyml-counterfactual
!pip install gdown python-dotenv

### Import Libraries

In [ ]:
import os
import pandas as pd
import eazyml as ez
from eazyml_counterfactual import (
        ez_cf_inference,
        ez_init        
)
import gdown

from dotenv import load_dotenv
load_dotenv()

### Initialize EazyML

In [ ]:
ez_init(os.getenv('EAZYML_ACCESS_KEY'))

## 1. Download the dataset and specify the outcome variable

### 1.1. Download dataset

In [ ]:
gdown.download_folder(id='1WvIOaIvS7hTlYSkeojYhnBpAWd8HWzDt')

###  1.2. Define dataset file and outcome variable

In [ ]:
# Defining file paths for training and test datasets and specifying the outcome variable
train_file = os.path.join('data', "Mobile Price Ternary - Train Data.xlsx")
test_file = os.path.join('data', "Mobile Price Ternary - Test Data.xlsx")
outcome = "price_range"

# Loading the training dataset and the test dataset
train_df = pd.read_excel(train_file)
test_df = pd.read_excel(test_file)

# Display the first few rows of the training DataFrame for inspection
ez.ez_display_df(train_df.head())

## 2. EazyML Modeling

### 2.1. Building model using the EazyML Modeling API

In [ ]:
# Clean and preprocess training data
train_df.drop(columns=['Unnamed: 0'], inplace=True)  # Remove unnecessary columns

# Define model parameters
model_options = {
    "model_type": "predictive",
}

# Build predictive model using EazyML API
build_model_response = ez.ez_build_model(train_df, outcome=outcome, options=model_options)

### 2.2. Feature Importance

In [ ]:
ez.ez_display_df(build_model_response['global_importance'])

### 2.3. Model Importance

In [ ]:
ez.ez_display_df(build_model_response['model_performance'])

### 2.4. Predict Using the Trained EazyML Model

In [ ]:
# Extract model information from the response dictionary
model_info = build_model_response["model_info"]

# Read test data from a CSV file into a pandas DataFrame
test_data = pd.read_excel(test_file)

# Make predictions using the model, requesting confidence scores and class probabilities
predicted_resp = ez.ez_predict(test_data, model_info, options={"confidence_score": True, "class_probability": True})

# Check if the prediction was successful
if predicted_resp['success']:
    print("Prediction successful")  
    predicted_df = predicted_resp['pred_df']  # Extract the predicted DataFrame
    ez.ez_display_df(predicted_df.head())  # Display the first few rows of the predicted DataFrame
else:
    print("Prediction failed")  
    print(predicted_resp['message'])  

## 3. EazyML Counterfactual Inference

### 3.1. Define Counterfactual Inference Configuration

In [ ]:
# Define the selected features for prediction
selected_features = ['sc_w', 'n_cores', 'mobile_wt', 'talk_time', 'ram', 'px_width', 'px_height', 
                     'battery_power', 'pc', 'fc', 'm_dep', 'int_memory', 'sc_h']

# Define variant (modifiable) features
invariants = []
variants = [feature for feature in selected_features if feature not in invariants]

# Define configurable parameters for counterfactual inference
cf_options = {   
    "variants": variants,  
    "outcome_ordinality": "1",  # Desired outcome 
    "train_data": train_file  
}

### 3.2. Perform Counterfactual Inference

In [ ]:
# Specify the index of the test record for counterfactual inference
test_index_no = 0  
test_data = predicted_df.loc[[test_index_no]]  

# Perform Inference 
result, optimal_transition_df = ez_cf_inference(
    test_data=test_data,  
    outcome=outcome,  
    selected_features=selected_features,  
    model_info=model_info,  
    options=cf_options  
)

### 3.3. Display Results

In [ ]:
# Summarizes whether an optimal transition was found and the improvement in outcome probability.
ez.ez_display_json(result)

In [ ]:
# Details the feature changes needed to achieve the optimal outcome.
ez.ez_display_df(optimal_transition_df)